In [1]:
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
import sqlite3
import json
from census import Census

In [2]:
# Read the Net_generation_for_all_sectors_monthly CSV file 
data = pd.read_csv('Net_generation_for_all_sectors_monthly.csv', sep=',', skiprows=4)

# Read the renewable_energy_consumption_source CSV file
renewable_energy_consumption = pd.read_csv('renewable_energy_consumption_source.csv')

transposed_data = data.transpose()

# Rename columns with values from the description row
transposed_data.columns = transposed_data.iloc[0]

# Drop the row after setting it as the header (optional)
transposed_data = transposed_data.drop('description')

# Delete 'Net generation for all sectors' and 'United States' columns
del transposed_data['Net generation for all sectors']
del transposed_data['United States']

# Display cleaned_data DataFrame the modified data
transposed_data.head()

description,United States : all fuels (utility-scale),United States : coal,United States : petroleum liquids,United States : petroleum coke,United States : natural gas,United States : other gases,United States : nuclear,United States : conventional hydroelectric,United States : other renewables,United States : wind,United States : all utility-scale solar,United States : geothermal,United States : biomass,United States : wood and wood-derived fuels,United States : other biomass,United States : hydro-electric pumped storage,United States : other,United States : all solar,United States : small-scale solar photovoltaic,United States : all utility-scale solar
units,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,NaN,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours,thousand megawatthours
source key,ELEC.GEN.ALL-US-99.M,ELEC.GEN.COW-US-99.M,ELEC.GEN.PEL-US-99.M,ELEC.GEN.PC-US-99.M,ELEC.GEN.NG-US-99.M,ELEC.GEN.OOG-US-99.M,ELEC.GEN.NUC-US-99.M,ELEC.GEN.HYC-US-99.M,ELEC.GEN.AOR-US-99.M,ELEC.GEN.WND-US-99.M,ELEC.GEN.SUN-US-99.M,ELEC.GEN.GEO-US-99.M,ELEC.GEN.BIO-US-99.M,ELEC.GEN.WWW-US-99.M,ELEC.GEN.WAS-US-99.M,ELEC.GEN.HPS-US-99.M,ELEC.GEN.OTH-US-99.M,ELEC.GEN.TSN-US-99.M,ELEC.GEN.DPV-US-99.M,ELEC.GEN.SUN-US-99.M
Jan 2018,373379.0,119284.0,5555.0,965.0,110442.0,1097.0,74649.0,25064.0,NaN,25599.0,3319.0,1341.0,5503.0,3686.0,1817.0,-547.0,1109.0,4938.0,1619.0,3319.0
Feb 2018,307058.0,82050.0,804.0,754.0,98675.0,1092.0,64790.0,24902.0,NaN,23189.0,3896.0,1274.0,4951.0,3235.0,1716.0,-315.0,994.0,5663.0,1766.0,3896.0
Mar 2018,321765.0,80626.0,830.0,642.0,106742.0,1158.0,67033.0,25861.0,NaN,26464.0,5056.0,1367.0,5369.0,3547.0,1822.0,-490.0,1108.0,7490.0,2434.0,5056.0


In [3]:
renewable_energy_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8931 entries, 0 to 8930
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   MSN           8931 non-null   object
 1   YYYYMM        8931 non-null   int64 
 2   Value         8931 non-null   object
 3   Column_Order  8931 non-null   int64 
 4   Description   8931 non-null   object
 5   Unit          8931 non-null   object
dtypes: int64(2), object(4)
memory usage: 418.8+ KB


In [4]:
renewable_energy_consumption.head()

,MSN,YYYYMM,Value,Column_Order,Description,Unit
0,WDPRBUS,194913,1549.262,1,Wood Energy Production,Trillion Btu
1,WDPRBUS,195013,1562.307,1,Wood Energy Production,Trillion Btu
2,WDPRBUS,195113,1534.669,1,Wood Energy Production,Trillion Btu
3,WDPRBUS,195213,1474.369,1,Wood Energy Production,Trillion Btu
4,WDPRBUS,195313,1418.601,1,Wood Energy Production,Trillion Btu


In [5]:
# Convert 'YYYYMM' column to string
renewable_energy_consumption['YYYYMM'] = renewable_energy_consumption['YYYYMM'].astype(str)

# Filter out rows where 'YYYYMM' ends with '13'
renewable_energy_consumption_df = renewable_energy_consumption[~renewable_energy_consumption['YYYYMM'].str.endswith('13')]

# Change 'YYYYMM' column name to 'Date'
renewable_energy_consumption_df.rename(columns={'YYYYMM':'Date'}, inplace=True)

# Convert the 'Date' column to a datetime format
renewable_energy_consumption_df['Date'] = pd.to_datetime(renewable_energy_consumption_df['Date'].astype(str), format='%Y%m' )
# df['date_column'] = pd.to_datetime(df['date_column'].astype(str), format='%Y%m%d')


renewable_energy_consumption_df.head()

/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_75225/636604446.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_energy_consumption_df.rename(columns={'YYYYMM':'Date'}, inplace=True)
/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_75225/636604446.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_energy_consumption_df['Date'] = pd.to_datetime(renewable_energy_consumption_df['Date'].astype(str), format='%Y%m' )


,MSN,Date,Value,Column_Order,Description,Unit
24,WDPRBUS,1973-01-01,129.63,1,Wood Energy Production,Trillion Btu
25,WDPRBUS,1973-02-01,117.194,1,Wood Energy Production,Trillion Btu
26,WDPRBUS,1973-03-01,129.763,1,Wood Energy Production,Trillion Btu
27,WDPRBUS,1973-04-01,125.462,1,Wood Energy Production,Trillion Btu
28,WDPRBUS,1973-05-01,129.624,1,Wood Energy Production,Trillion Btu


In [6]:
renewable_energy_consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7956 entries, 24 to 8929
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   MSN           7956 non-null   object        
 1   Date          7956 non-null   datetime64[ns]
 2   Value         7956 non-null   object        
 3   Column_Order  7956 non-null   int64         
 4   Description   7956 non-null   object        
 5   Unit          7956 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 435.1+ KB


In [7]:
# Filter data for all dates from year >= 2018
filtered_df = renewable_energy_consumption_df[renewable_energy_consumption_df['Date'].dt.year >= 2018]
filtered_df

,MSN,Date,Value,Column_Order,Description,Unit
609,WDPRBUS,2018-01-01,202.355,1,Wood Energy Production,Trillion Btu
610,WDPRBUS,2018-02-01,184.22,1,Wood Energy Production,Trillion Btu
611,WDPRBUS,2018-03-01,198.708,1,Wood Energy Production,Trillion Btu
612,WDPRBUS,2018-04-01,187.922,1,Wood Energy Production,Trillion Btu
613,WDPRBUS,2018-05-01,197.119,1,Wood Energy Production,Trillion Btu
...,...,...,...,...,...,...
8925,RETCBUS,2023-08-01,703.185,13,Total Renewable Energy Consumption,Trillion Btu
8926,RETCBUS,2023-09-01,651.931,13,Total Renewable Energy Consumption,Trillion Btu
8927,RETCBUS,2023-10-01,690.408,13,Total Renewable Energy Consumption,Trillion Btu
8928,RETCBUS,2023-11-01,664.473,13,Total Renewable Energy Consumption,Trillion Btu
